# PROJECT 2 - PROBLEM 2

In [ ]:
# Author: Dani Gunawan
import numpy as np

In [ ]:
# connectivity matrix
conn_matrix = np.array([[ 0, 0, -1,  0],
                        [ 1, 0, -1, -1],
                        [ 0, 1,  0,  0],
                        [-1, 1,  1,  0]])

# state space s1-s16
s = np.array([[0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0], [0, 0, 1, 1], 
              [0, 1, 0, 0], [0, 1, 0, 1], [0, 1, 1, 0], [0, 1, 1, 1],
              [1, 0, 0, 0], [1, 0, 0, 1], [1, 0, 1, 0], [1, 0, 1, 1],
              [1, 1, 0, 0], [1, 1, 0, 1], [1, 1, 1, 0], [1, 1, 1, 1]])

# action
a = np.array([[0, 0, 0, 0], [1, 0, 0, 0], [0, 0, 0, 1]])

max = 16

In [ ]:
def get_transition_prob(s_row, s_col, c_mat, a, p):
  # calculate L-1 norm
  cs = np.dot(c_mat, np.transpose(s_row))
  cs_bar = []
  for n in cs:
    if n <= 0:
      cs_bar.append(0)
    else:
      cs_bar.append(1)

  l1 = np.sum(np.abs(s_col - np.logical_xor(cs_bar, a)))

  prob = np.power(p, l1) * np.power((1-p), (4-l1))

  return prob


In [ ]:
def generate_transition_matrix(p, states, actions, c_mat):
  
  t_matrix = []

  for act in actions:
    m = np.zeros((max, max))
    for i in range(0,max):
      for j in range(0, max):
        m[i][j] = get_transition_prob(states[i], states[j], c_mat, act, p)
    #print(np.sum(m, axis=1)) # the sum of transition matrix row should be 1
    t_matrix.append(m)

  return t_matrix

In [ ]:
def generate_reward_matrix(states, actions):

  r_matrix = []

  for act in actions:
    m = np.zeros((max, max))
    for i in range(0,max):
      for j in range(0, max):
        m[i][j] = (5*states[j][0])+(5*states[j][1])+(5*states[j][2])+(5*states[j][3])-np.sum(act)
    r_matrix.append(m)

  return r_matrix

In [ ]:
def generate_reward_action(t_mat, r_mat, actions):

  rs_a = []

  vec_nx1 = np.full(max, 1) 

  for i in range(0, len(actions)):
    rs_a.append(np.dot(np.multiply(t_mat[i], r_mat[i]), np.transpose(vec_nx1)))

  return rs_a

In [ ]:
def value_iteration_matrix(actions, t_mat, rsa, g, t):
  v = []
  v.append(np.zeros(max))
  v_star = 0
  n = 0  
  
  while (True):
    #print("Iteration: " + str(n))
    #print(v[n])
    tmp_v = []
    for i in range(0, len(actions)):
      tmp_v.append(rsa[i]+(g*np.dot(t_mat[i], v[n])))
    
    v.append(np.amax(tmp_v, axis=0))
  
    dv_max = np.max(np.absolute(np.subtract(v[n+1], v[n])))
    #print(dv_max)
    
    if dv_max < t:
      v_star = v[n+1]
      break
    else:
      n += 1
  
  # calculate optimal policy
  pi = []
  for i in range(0, len(actions)):
    pi.append(rsa[i]+(g*np.dot(t_mat[i], v_star)))
  
  p_star = np.argmax(pi, axis=0)

  return np.array(p_star), np.array(v_star)

In [ ]:
def performance_evaluation(pi, t_m, no_control=False):

  # performance evaluation
  avg = []
  
  print('+----+-----------+')
  print('| #i |   State   |')
  print('+----+-----------+')

  for e in range(0, 100):
    idx = np.random.randint(0, max) # index state, initially set random

    d = [] # trajectory
    #idx
    for i in range(0, 200):

      # this index is obtained from optimal policy, 
      # to determine transition matrix associated with the optimal policy (action)
      if no_control==True:
        act_idx = 0 # set to policy a1
      else:
        act_idx = pi[idx] 

      ## find next state
      # get all possibilities in transition matrix, row state idx
      # then, apply cumulative sum 
      prob = np.cumsum(t_m[act_idx][idx]) 

      # generate random values uniformly
      r = np.random.uniform(0, 1)
      #print(r)
      tmp_idx = 0
      for pb in prob:
        if r<=pb:
          #if e == 0 and i < 30:
          #  print('r = ' + str(np.round(r, 10)) + ' idx =  ' + str(tmp_idx) + ' ', end='')
          break #stop
        else:
          tmp_idx += 1

      idx = tmp_idx # new state index

      # sum |sk|
      # print(np.sum(s[idx]))
      # print first n values
      if e == 0 and i < 15:
        print('| ' + str(i+1).rjust(2, '0') + ' | ', end='')
        print(s[idx], end='')
        print(' |')
      d.append(np.sum(s[idx]))
      
      #print(d)
      #print(np.shape(d))
      #print("Avg")
      #print(np.average(d))

    #print(np.shape(d))
    avg.append(np.average(d))
    #print(np.shape(avg))

  print('+----+-----------+')

  return np.average(avg)

### PROBLEM 2 - a (MATRIX FORM - VALUE ITERATION)

𝑝=0.05, 𝜃=0.01, 𝛾=0.95, all-zero initial state values

In [ ]:
p = 0.05
g = 0.95
t = 0.01

t_m = generate_transition_matrix(p, s, a, conn_matrix)

r_m = generate_reward_matrix(s, a)

r_a = generate_reward_action(t_m, r_m, a)

pi, vi = value_iteration_matrix(a, t_m, r_a, g, t)

print("Optimal Policy:")
print(pi)
#print("V star:")
#print(vi)

av = performance_evaluation(pi, t_m)

print("\nNo control: ")

av_noc = performance_evaluation(pi, t_m, True) # no control

print("AvgA under optimal control policy: {}".format(av))

print("AvgA under no control policy: {}".format(av_noc))

Optimal Policy:
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
+----+-----------+
| #i |   State   |
+----+-----------+
| 01 | [1 0 1 1] |
| 02 | [0 0 0 0] |
| 03 | [1 0 0 0] |
| 04 | [1 1 0 0] |
| 05 | [1 1 1 0] |
| 06 | [1 0 1 1] |
| 07 | [1 0 0 0] |
| 08 | [1 1 0 0] |
| 09 | [1 0 1 0] |
| 10 | [1 0 0 0] |
| 11 | [1 1 0 0] |
| 12 | [1 1 1 0] |
| 13 | [1 0 1 1] |
| 14 | [1 0 0 0] |
| 15 | [0 1 0 0] |
+----+-----------+

No control: 
+----+-----------+
| #i |   State   |
+----+-----------+
| 01 | [0 0 0 0] |
| 02 | [0 0 0 0] |
| 03 | [0 0 1 0] |
| 04 | [0 0 0 1] |
| 05 | [0 0 0 0] |
| 06 | [0 0 0 0] |
| 07 | [0 0 0 0] |
| 08 | [0 0 1 0] |
| 09 | [0 0 0 1] |
| 10 | [0 0 0 0] |
| 11 | [0 0 0 0] |
| 12 | [0 0 0 0] |
| 13 | [0 0 0 0] |
| 14 | [0 0 0 0] |
| 15 | [0 0 1 0] |
+----+-----------+
AvgA under optimal control policy: 2.1506
AvgA under no control policy: 0.46855


In [ ]:
prob = np.cumsum(t_m[0][0])
for i in range(16):
  print('idx: ' + str(i) + ' ' + str(np.round(t_m[0][0][i], 10)))
  i += 1

print('\n')

for i in range(16):
  print(np.round(prob[i], 10))
  i += 1

idx: 0 0.81450625
idx: 1 0.04286875
idx: 2 0.04286875
idx: 3 0.00225625
idx: 4 0.04286875
idx: 5 0.00225625
idx: 6 0.00225625
idx: 7 0.00011875
idx: 8 0.04286875
idx: 9 0.00225625
idx: 10 0.00225625
idx: 11 0.00011875
idx: 12 0.00225625
idx: 13 0.00011875
idx: 14 0.00011875
idx: 15 6.25e-06


0.81450625
0.857375
0.90024375
0.9025
0.94536875
0.947625
0.94988125
0.95
0.99286875
0.995125
0.99738125
0.9975
0.99975625
0.999875
0.99999375
1.0


### PROBLEM 2 - b (MATRIX FORM - VALUE ITERATION)

b.1 𝑝=0.2, 𝜃=0.01, 𝛾=0.95

In [ ]:
p = 0.2
g = 0.95
t = 0.01

t_m = generate_transition_matrix(p, s, a, conn_matrix)

r_m = generate_reward_matrix(s, a)

r_a = generate_reward_action(t_m, r_m, a)

pi, vi = value_iteration_matrix(a, t_m, r_a, g, t)

print("Optimal Policy:")
print(pi)
#print("V star:")
#print(vi)

av = performance_evaluation(pi, t_m)

print("AvgA under optimal control policy: {}".format(av))


Optimal Policy:
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
+----+-----------+
| #i |   State   |
+----+-----------+
| 01 | [0 0 0 0] |
| 02 | [0 1 0 0] |
| 03 | [0 0 1 1] |
| 04 | [1 0 0 1] |
| 05 | [1 1 0 0] |
| 06 | [1 1 1 0] |
| 07 | [1 1 1 1] |
| 08 | [0 0 1 1] |
| 09 | [1 0 0 1] |
| 10 | [0 0 0 0] |
| 11 | [1 0 0 0] |
| 12 | [0 0 0 0] |
| 13 | [1 0 0 0] |
| 14 | [1 1 0 0] |
| 15 | [1 1 1 0] |
+----+-----------+
AvgA under optimal control policy: 1.98125


In [ ]:
for i in range(16):
  print('idx: ' + str(i) + ' ' + str(np.round(t_m[0][0][i], 10)))
  i += 1

idx: 0 0.4096
idx: 1 0.1024
idx: 2 0.1024
idx: 3 0.0256
idx: 4 0.1024
idx: 5 0.0256
idx: 6 0.0256
idx: 7 0.0064
idx: 8 0.1024
idx: 9 0.0256
idx: 10 0.0256
idx: 11 0.0064
idx: 12 0.0256
idx: 13 0.0064
idx: 14 0.0064
idx: 15 0.0016


b.2 𝑝=0.45, 𝜃=0.01, 𝛾=0.95

In [ ]:
p = 0.45
g = 0.95
t = 0.01

t_m = generate_transition_matrix(p, s, a, conn_matrix)

r_m = generate_reward_matrix(s, a)

r_a = generate_reward_action(t_m, r_m, a)

pi, vi = value_iteration_matrix(a, t_m, r_a, g, t)

print("Optimal Policy:")
print(pi)
#print("V star:")
#print(vi)

av = performance_evaluation(pi, t_m)

print("AvgA under optimal control policy: {}".format(av))


Optimal Policy:
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
+----+-----------+
| #i |   State   |
+----+-----------+
| 01 | [0 1 1 0] |
| 02 | [1 0 1 0] |
| 03 | [1 0 1 0] |
| 04 | [1 0 0 1] |
| 05 | [0 1 1 0] |
| 06 | [1 1 0 0] |
| 07 | [0 0 1 1] |
| 08 | [0 0 0 0] |
| 09 | [0 0 0 1] |
| 10 | [0 0 0 1] |
| 11 | [0 1 0 1] |
| 12 | [1 0 1 0] |
| 13 | [1 1 1 1] |
| 14 | [1 0 1 1] |
| 15 | [1 0 1 0] |
+----+-----------+
AvgA under optimal control policy: 1.9112499999999997


In [ ]:
for i in range(16):
  print('idx: ' + str(i) + ' ' + str(np.round(t_m[0][0][i], 10)))
  i += 1

idx: 0 0.09150625
idx: 1 0.07486875
idx: 2 0.07486875
idx: 3 0.06125625
idx: 4 0.07486875
idx: 5 0.06125625
idx: 6 0.06125625
idx: 7 0.05011875
idx: 8 0.07486875
idx: 9 0.06125625
idx: 10 0.06125625
idx: 11 0.05011875
idx: 12 0.06125625
idx: 13 0.05011875
idx: 14 0.05011875
idx: 15 0.04100625


### PROBLEM 2 - c (MATRIX FORM - VALUE ITERATION)

𝑝=0.05, 𝜃=0.01, 𝛾=0.95, all-zero initial state values

a1 = [0, 0, 0, 0], a2 = [1, 0, 0, 0], a3 = [0, 1, 0, 0], a4 = [0, 0, 1, 0], a5 = [0, 0, 0, 1]

In [ ]:
# action
a_c = np.array([[0, 0, 0, 0], [1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])

p = 0.05
g = 0.95
t = 0.01

t_m_c = generate_transition_matrix(p, s, a_c, conn_matrix)

r_m_c = generate_reward_matrix(s, a_c)

r_a_c = generate_reward_action(t_m_c, r_m_c, a_c)

pi_c, vi_c = value_iteration_matrix(a_c, t_m_c, r_a_c, g, t)

print("Optimal Policy:")
print(pi_c)
#print("V star:")
#print(vi)

av_c = performance_evaluation(pi_c, t_m_c)

av_noc_c = performance_evaluation(pi_c, t_m_c, True) # no control

print("AvgA under optimal control policy: {}".format(av_c))

print("AvgA under no control policy: {}".format(av_noc_c))


Optimal Policy:
[2 2 2 2 2 2 2 2 3 2 2 2 4 2 2 2]
+----+-----------+
| #i |   State   |
+----+-----------+
| 01 | [0 1 0 1] |
| 02 | [0 1 1 1] |
| 03 | [0 1 1 1] |
| 04 | [0 1 1 1] |
| 05 | [0 1 1 1] |
| 06 | [0 1 1 1] |
| 07 | [0 1 1 1] |
| 08 | [0 1 1 1] |
| 09 | [0 1 1 1] |
| 10 | [0 1 1 1] |
| 11 | [0 1 1 1] |
| 12 | [0 1 1 1] |
| 13 | [0 1 1 1] |
| 14 | [0 1 1 1] |
| 15 | [0 1 1 0] |
+----+-----------+
+----+-----------+
| #i |   State   |
+----+-----------+
| 01 | [0 0 0 0] |
| 02 | [0 0 0 1] |
| 03 | [0 1 1 0] |
| 04 | [0 0 1 1] |
| 05 | [0 0 0 1] |
| 06 | [0 0 1 0] |
| 07 | [0 0 0 1] |
| 08 | [0 0 0 0] |
| 09 | [0 0 0 0] |
| 10 | [1 0 0 0] |
| 11 | [1 1 0 0] |
| 12 | [0 1 1 0] |
| 13 | [0 0 1 1] |
| 14 | [0 0 0 1] |
| 15 | [0 0 0 0] |
+----+-----------+
AvgA under optimal control policy: 2.8404000000000003
AvgA under no control policy: 0.49080000000000007


In [ ]:
for i in range(16):
  print('idx: ' + str(i) + ' ' + str(np.round(t_m_c[0][0][i], 10)))
  i += 1

idx: 0 0.81450625
idx: 1 0.04286875
idx: 2 0.04286875
idx: 3 0.00225625
idx: 4 0.04286875
idx: 5 0.00225625
idx: 6 0.00225625
idx: 7 0.00011875
idx: 8 0.04286875
idx: 9 0.00225625
idx: 10 0.00225625
idx: 11 0.00011875
idx: 12 0.00225625
idx: 13 0.00011875
idx: 14 0.00011875
idx: 15 6.25e-06


### PROBLEM 2 - d (MATRIX FORM - POLICY ITERATION)

𝑝=0.05, 𝜃=0.01, 𝛾=0.95, all-zero initial state values

In [ ]:
def generate_matrix_pi(policy, t_mat):
  m_pi = []
  i = 0
  for pol in policy:
    m_pi.append(t_mat[pol][i])
    i += 1

  return np.array(m_pi)

In [ ]:
def generate_reward_pi(policy, r_sa):
  r_pi = []
  i = 0
  for pol in policy:
    r_pi.append(r_sa[pol][i])
    i += 1

  return np.array(r_pi)


In [ ]:
def calculate_v_pi(m_pi, r_pi, g):

  i_mat = np.identity(max)

  v_pi = np.dot(np.linalg.inv(np.subtract(i_mat, g*m_pi)), r_pi)

  return v_pi

In [ ]:
def policy_iteration_matrix(t_mat, rsa, g):
  n = 0
  pi = []
  pi.append(np.zeros(max, dtype=np.int32))
  p_star = 0

  while (True):
    #print(n)
    #print(pi[n])
    m_pi = generate_matrix_pi(pi[n], t_mat)
    r_pi = generate_reward_pi(pi[n], rsa)
    v_pi = calculate_v_pi(m_pi, r_pi, g)
    
    tmp_pi = []
    for i in range(0, len(a)):
      tmp_pi.append(rsa[i]+(g*np.dot(t_mat[i], v_pi)))
    
    #print(np.argmax(tmp_pi, axis=0))
    pi.append(np.argmax(tmp_pi, axis=0))

    if (np.array_equal(pi[n+1], pi[n])):
      p_star = pi[n+1]
      break
    else:
      n += 1

  return p_star

In [ ]:
p = 0.05
g = 0.95
t = 0.01

t_m = generate_transition_matrix(p, s, a, conn_matrix)

r_m = generate_reward_matrix(s, a)

r_a = generate_reward_action(t_m, r_m, a)

pi = policy_iteration_matrix(t_m, r_a, g)

print("Optimal Policy:")
print(pi)
#print("V star:")
#print(vi)

av = performance_evaluation(pi, t_m)

av_noc = performance_evaluation(pi, t_m, True) # no control

print("AvgA under optimal control policy: {}".format(av))

print("AvgA under no control policy: {}".format(av_noc))


Optimal Policy:
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
+----+-----------+
| #i |   State   |
+----+-----------+
| 01 | [1 0 1 1] |
| 02 | [1 0 0 0] |
| 03 | [1 1 0 0] |
| 04 | [1 1 1 0] |
| 05 | [1 0 1 0] |
| 06 | [1 0 0 0] |
| 07 | [1 1 0 0] |
| 08 | [1 1 1 0] |
| 09 | [1 0 1 1] |
| 10 | [1 0 0 0] |
| 11 | [1 1 0 0] |
| 12 | [1 1 1 0] |
| 13 | [1 0 1 1] |
| 14 | [1 0 0 0] |
| 15 | [1 1 0 0] |
+----+-----------+
+----+-----------+
| #i |   State   |
+----+-----------+
| 01 | [0 0 1 1] |
| 02 | [0 0 0 1] |
| 03 | [0 0 0 0] |
| 04 | [0 0 0 0] |
| 05 | [0 0 1 0] |
| 06 | [0 0 0 1] |
| 07 | [0 0 0 0] |
| 08 | [0 0 0 1] |
| 09 | [0 0 0 0] |
| 10 | [0 0 0 1] |
| 11 | [0 0 0 0] |
| 12 | [0 0 0 1] |
| 13 | [1 0 0 0] |
| 14 | [0 1 0 0] |
| 15 | [1 0 0 1] |
+----+-----------+
AvgA under optimal control policy: 2.15675
AvgA under no control policy: 0.47875
